In [ ]:
import sys, os
import numpy as np
from tqdm import tqdm
module_path = os.path.abspath(os.path.join(".."))
sys.path.append(module_path + "/src")

from config.Config import Config
from models.TrainModel import TrainModel
from shared import utils
import pandas as pd
from data.Data import Data
from shared import gpu_selection
from shared import plots
gpu_selection.autoselect_gpu()
import matplotlib.pyplot as plt
import elbo_calculator

In [ ]:
N_EXPERIMENTS = 10

output_dir = "../output/"
#dataset_dirs = ["Lenze"]
#schedules = ["Lenze_batch64_f_only"]
#dataset_dirs = ["Lenze", "Ecg_Heart"]
dataset_dirs = ["ti-rvae-july/lenze", "ti-rvae/ecg", "ti-rvae/ett"]
#dataset_dirs = ["ti-rvae/ett"]
#schedules = ["Lenze_batch64_f_only", "Lenze_batch64_2f_1b"]

df_results = pd.DataFrame()
for dataset_dir in dataset_dirs:
    subdir_path = output_dir + dataset_dir

    schedule_results = dict()
    for schedule_dir in os.listdir(subdir_path):
        schedule_dir_path = subdir_path + "/" +schedule_dir

        results = list()
        for dir in (os.listdir(schedule_dir_path)):
            subdir = schedule_dir_path + "/" + dir
            print(f"processing {subdir}")
            config = Config()
            config.select_output_config_by_args(subdir)
            config.network_components.loss_fnc = "mse_kl_no_batch_avg"
            data =  Data(config)
            test_data = data.get_sliding_windows("train")
            train_model = TrainModel(config)
            train_model.compile_model()
            
            chunks = np.array_split(test_data, 10)
            elbos_list = list()
            for chunk in chunks:
                elbos = train_model.model.call_and_loss(chunk, chunk)[0].numpy()
                elbos_list+=list(elbos)
            #elbos = train_model.model.evaluate(test_data)[0]
            elbos = np.asarray(elbos_list)
            elbos_reduced = elbo_calculator.get_elbo_reduced(elbos,
                                                             config.hyperparameters.time_window,
                                                             config.hyperparameters.n_channels,
                                                             config.hyperparameters.alpha,
                                                             config.hyperparameters.beta)
            elbos_mean = np.mean(elbos_reduced)
            results.append(elbos_mean)
        filled_results = N_EXPERIMENTS * [np.nan]
        filled_results[0:len(results)] = results
        if (len(filled_results) != len(results)):
            print(f"Warning:  subdir_path not enough trained")
        splitted = schedule_dir_path.split("/")
        df_results[splitted[-2]+splitted[-1]] = filled_results  


In [ ]:
df_results.head(n=10)

In [ ]:
for col in df_results.columns:
    mean = df_results[col].mean()
    std = df_results[col].std()
    print(f"{col}: {mean} +- {std}")